# Imports

In [ ]:
!pip3 install wandb matplotlib torch torchviz torchvision torchsummary torchviz weave nbformat netron onnx roboflow scikit-learn roboflow netron ultralytics --quiet

import os
from os import path
import json
import random
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
from PIL import Image, ImageDraw
from torchvision import datasets, transforms, utils
import wandb
import netron
from roboflow import Roboflow
import ultralytics

# Utils

In [ ]:
def show_image(img):
    plt.imshow(transforms.functional.to_pil_image(img))
    plt.show()

def collate_fn(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, dim=0)
    return images, labels

def show_image_with_labels(image, labels, class_names=None):
    image_np = image.permute(1, 2, 0).numpy()
    h, w, _ = image_np.shape

    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.imshow(image_np)

    for label in labels:
        class_id, x_center, y_center, bw, bh = label.tolist()
        x = (x_center - bw / 2) * w
        y = (y_center - bh / 2) * h
        box_w = bw * w
        box_h = bh * h
        rect = patches.Rectangle((x, y), box_w, box_h, linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        if class_names:
            class_text = class_names[int(class_id)]
        else:
            class_text = str(int(class_id))
        ax.text(x, y - 5, class_text, color='white', fontsize=12,bbox=dict(facecolor='red', alpha=0.5, pad=2))
    plt.axis('off')
    plt.show()

random.seed(42)
np.random.seed(42)

# Dataset

In [ ]:
import os
from PIL import Image
from os import path
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class MinecraftV1(Dataset):
    def __init__(self, root, train=True, valid=False, transform=None):
        super().__init__()
        self.root = root
        self.train = train
        self.valid = valid
        self.transform = transform

        if train:
            self.data_path = os.path.join(root, 'train')
        elif valid:
            self.data_path = os.path.join(root, 'valid')
        else:
            self.data_path = os.path.join(root, 'test')

        self.images_path = os.path.join(self.data_path, 'images')
        self.labels_path = os.path.join(self.data_path, 'labels')
        self.data_images = []
        self.data_labels = []
        image_files = sorted(os.listdir(self.images_path))
        label_files = sorted(os.listdir(self.labels_path))
        for image_file in image_files:
            image_path = os.path.join(self.images_path, image_file)
            self.data_images.append(image_path)

        for label_file in label_files:
            label_path = os.path.join(self.labels_path, label_file)
            self.data_labels.append(label_path)

    def __len__(self):
        return len(self.data_images)

    def __getitem__(self, idx):
        image_path = self.data_images[idx]
        label_path = self.data_labels[idx]

        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        with open(label_path, 'r') as file:
            lines = file.readlines()
        labels = [list(map(float, line.strip().split())) for line in lines]
        labels = torch.tensor(labels, dtype=torch.float32)
        return image, labels

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.05):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn_like(tensor) * self.std + self.mean

    def __repr__(self):
        return f"{self.__class__.__name__}(mean={self.mean}, std={self.std})"

In [ ]:
from roboflow import Roboflow

classes_types = {
    0: 'gold_ore',
    1: 'iron_ore',
    2: 'diamond_ore',
    3: 'redstone_ore',
    4: 'deepslate_iron_ore'
}
classes_number = len(classes_types)

dataset_dir = "minecraft-ore-1"
rf = Roboflow(api_key="bVvy50uMbWp85HBSLUkm")
project = rf.workspace("oblig10").project("minecraft-ore")
version = project.version(1)
dataset = version.download("yolov5")

file_path_label = "minecraft-ore-1/valid/labels/2024-04-25_19-38-10_png_jpg.rf.627bb52ce40ad0431564b93df2aa900f.txt"
file_path_image = "minecraft-ore-1/valid/images/2024-04-25_19-38-10_png_jpg.rf.627bb52ce40ad0431564b93df2aa900f.jpg"
if os.path.exists(file_path_label):
    os.remove(file_path_label)
    print(f"Deleted: {file_path_label}")
if os.path.exists(file_path_image):
    os.remove(file_path_image)
    print(f"Deleted: {file_path_image}")

basic_transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

augmented_transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.05),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    AddGaussianNoise(0., 0.03)
])

mc_train = MinecraftV1(root=os.path.join(os.getcwd(), "minecraft-ore-1"), transform=augmented_transform)
mc_test = MinecraftV1(root=os.path.join(os.getcwd(), "minecraft-ore-1"), train=False, transform=basic_transform)
mc_valid = MinecraftV1(root=os.path.join(os.getcwd(), "minecraft-ore-1"), train=False, valid=True, transform=basic_transform)
trainloader = data.DataLoader(mc_train, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=0)
validloader = data.DataLoader(mc_valid, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=0)
testloader = data.DataLoader(mc_test, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=0)

## Visualization

In [ ]:
images, labels = next(iter(trainloader))
for i in range(4):
    show_image_with_labels(images[i], labels[i], class_names=classes_types)

# Training

In [ ]:
from ultralytics import YOLO
import torch

def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

device = get_device()
print(f"Using device: {device}")

model = YOLO('yolov5m.yaml')

results = model.train(
    data="minecraft-ore-1/data.yaml",
    epochs=20,
    imgsz=640,
    batch=4,
    workers=2,
    device=0 if torch.cuda.is_available() else "cpu"
)
model.save("yolov5m_trained.pt")

# Results

In [ ]:
def load_model(model, model_path):
    """Load a model from a .pth file."""
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model


In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

model_path = "yolov5m_trained.pt"
model = YOLO(model_path)
mc_test = MinecraftV1(root=os.path.join(os.getcwd(), "minecraft-ore-1"), train=False, transform=basic_transform)
testloader = data.DataLoader(mc_test, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=0)

img_idx = random.randint(0, len(mc_test) - 1)
img, _ = mc_test[img_idx]
results = model(img.unsqueeze(0))

# Get predictions for the first image in the batch
pred = results[0]
boxes = pred.boxes.xyxy.cpu().numpy()  # (x1, y1, x2, y2)
scores = pred.boxes.conf.cpu().numpy()
classes = pred.boxes.cls.cpu().numpy()

# Convert tensor image to numpy for display
img_np = img.permute(1, 2, 0).cpu().numpy()
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.imshow(img_np)

for box, cls, score in zip(boxes, classes, scores):
    x1, y1, x2, y2 = box
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)
    class_name = classes_types.get(int(cls), str(int(cls)))
    ax.text(x1, y1 - 5, f"{class_name} ({score:.2f})", color='white', fontsize=12,
            bbox=dict(facecolor='red', alpha=0.5, pad=2))
plt.axis('off')
plt.show()

# Evaluate

In [ ]:
results = model.val(
    data="minecraft-ore-1/data.yaml",
    split="val",
    imgsz=640,
    batch=4,
    device=0 if torch.cuda.is_available() else "cpu"
)

for class_id in range(len(classes_types)):
    precision, recall, f1, ap50 = results.class_result(class_id)
    class_name = classes_types.get(class_id, str(class_id))
    print(f"{class_name}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}, mAP50={ap50:.3f}")
